In [93]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [94]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Dropout

Read CSV create, and clean dataframe

In [95]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Teams.csv")

mlb_teams

,yearID,lgID,G,G.1,W,L,WSWin,R,AB,H,...,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2014,NL,162,81,64,98,N,615,5552,1379,...,47,118,398,1165,86,4.26,2,4,35,Arizona Diamondbacks
1,2014,NL,162,81,79,83,N,573,5468,1316,...,22,123,472,1369,95,3.38,5,13,54,Atlanta Braves
2,2014,AL,162,81,96,66,N,705,5596,1434,...,16,211,401,1285,44,3.43,3,13,53,Baltimore Orioles
3,2014,AL,162,81,71,91,N,634,5551,1355,...,20,123,535,1337,63,4.01,3,7,36,Boston Red Sox
4,2014,NL,162,81,73,89,N,614,5508,1315,...,31,157,442,1477,65,3.91,1,11,37,Chicago Cubs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1969,NL,162,81,88,74,N,725,5626,1557,...,52,119,454,944,74,3.61,39,9,33,Pittsburgh Pirates
1225,1969,NL,162,81,52,110,N,468,5357,1203,...,42,99,423,1143,45,4.24,16,9,25,San Diego Padres
1226,1969,NL,162,81,90,72,N,713,5474,1325,...,28,136,711,1054,71,3.26,71,15,17,San Francisco Giants
1227,1969,NL,162,80,87,75,N,595,5536,1403,...,44,90,503,876,87,2.94,63,12,26,St. Louis Cardinals


In [96]:
#Get list of dataframe columns

mlb_teams.columns

Index(['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name'],
      dtype='object')

In [97]:
#Select columns for dataframe


mlb_df = mlb_teams[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]
mlb_df.head(20)

,yearID,W,L,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2014,64,98,N,615,5552,1379,259,47,118,398,1165,86,4.26,2,4,35,Arizona Diamondbacks
1,2014,79,83,N,573,5468,1316,240,22,123,472,1369,95,3.38,5,13,54,Atlanta Braves
2,2014,96,66,N,705,5596,1434,264,16,211,401,1285,44,3.43,3,13,53,Baltimore Orioles
3,2014,71,91,N,634,5551,1355,282,20,123,535,1337,63,4.01,3,7,36,Boston Red Sox
4,2014,73,89,N,614,5508,1315,270,31,157,442,1477,65,3.91,1,11,37,Chicago Cubs
5,2014,73,89,N,660,5543,1400,279,32,155,417,1362,85,4.29,3,6,36,Chicago White Sox
6,2014,76,86,N,595,5395,1282,254,20,131,415,1252,122,3.59,5,13,44,Cincinnati Reds
7,2014,85,77,N,669,5575,1411,284,23,142,504,1189,104,3.56,6,15,40,Cleveland Indians
8,2014,66,96,N,755,5612,1551,307,41,186,397,1281,85,4.84,1,4,24,Colorado Rockies
9,2014,90,72,N,757,5630,1557,325,26,155,443,1144,106,4.01,5,8,41,Detroit Tigers


In [98]:
#Use getdummies to convert strings

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

data_bin.head()

,yearID,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name,WSWin_Y
0,2014,64,98,615,5552,1379,259,47,118,398,1165,86,4.26,2,4,35,Arizona Diamondbacks,0
1,2014,79,83,573,5468,1316,240,22,123,472,1369,95,3.38,5,13,54,Atlanta Braves,0
2,2014,96,66,705,5596,1434,264,16,211,401,1285,44,3.43,3,13,53,Baltimore Orioles,0
3,2014,71,91,634,5551,1355,282,20,123,535,1337,63,4.01,3,7,36,Boston Red Sox,0
4,2014,73,89,614,5508,1315,270,31,157,442,1477,65,3.91,1,11,37,Chicago Cubs,0


In [99]:
#Set X(data) and y(team)variables

data = data_bin.drop("name", axis=1)
outcome = data_bin["name"]
print(data.shape, outcome.shape)

(1229, 17) (1229,)


In [100]:
data_bin.count()

yearID     1229
W          1229
L          1229
R          1229
AB         1229
H          1229
2B         1229
3B         1229
HR         1229
BB         1229
SO         1229
SB         1229
ERA        1229
CG         1229
SHO        1229
SV         1229
name       1229
WSWin_Y    1229
dtype: int64

Preprocess data

In [101]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

In [102]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data, outcome, random_state=42)
X_train

,yearID,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
997,1978,95,67,727,5437,1435,251,27,149,610,818,137,3.12,46,16,38,0
39,2013,93,69,796,5735,1625,292,23,176,531,1073,35,3.61,3,12,39,0
30,2013,81,81,685,5676,1468,302,31,130,519,1142,62,3.92,6,7,38,0
242,2006,70,92,768,5610,1556,288,20,164,474,878,121,5.35,5,9,35,0
756,1987,76,85,720,5583,1475,244,33,209,504,1064,109,4.55,11,5,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1976,102,60,857,5702,1599,271,63,141,681,902,210,3.51,33,12,45,1
1095,1974,81,81,653,5489,1441,222,41,110,471,864,108,3.46,36,18,18,0
1130,1973,60,102,548,5457,1330,198,26,112,401,966,88,4.16,34,10,23,0
860,1983,71,91,701,5512,1436,272,42,140,470,868,84,4.08,9,10,42,0


In [140]:
#Scale data using MinMaxScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train

997      Los Angeles Dodgers
39            Detroit Tigers
30      Arizona Diamondbacks
242        Baltimore Orioles
756             Chicago Cubs
                ...         
1044         Cincinnati Reds
1095          Houston Astros
1130        San Diego Padres
860             Chicago Cubs
1126        New York Yankees
Name: name, Length: 921, dtype: object

In [104]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_test

array([12, 21,  6, 18,  7, 14,  7, 27, 13, 10,  0, 20,  1, 18,  5, 28, 13,
        6, 35, 32, 22, 26, 21,  6, 29, 13, 13, 11, 23, 16, 14, 23, 27, 28,
       20,  7, 16, 21, 33,  7, 27, 14, 13, 24, 22, 27, 24, 20, 30,  3, 14,
       27,  7, 17, 24, 19,  4, 16, 27, 21, 21,  3, 18,  2,  3, 18,  2,  5,
       19, 18, 28,  9, 22,  2, 16, 14, 33,  4, 20, 29, 33,  7, 13,  6, 27,
        4,  6, 33, 25, 18, 13,  6, 25, 12, 28, 28,  3, 11, 14, 24, 29, 24,
       26, 26, 25,  2, 21, 28, 19, 22,  4, 34, 16, 28,  6,  3, 22, 31, 31,
        4,  8, 23,  5, 13, 33,  8,  2,  3,  7, 25, 33,  9,  9, 10, 20,  4,
       26, 20, 23,  8,  2, 20, 28, 25, 25, 16, 23, 13, 21, 19, 28, 26, 32,
        6, 32, 18, 25, 23, 23, 13, 32,  7,  5,  3,  3, 14,  2,  4,  6, 33,
        1,  7,  4,  8,  9, 21, 18, 33, 12, 22, 13,  6, 10, 24, 22, 26, 19,
       18,  9, 22,  8, 20, 27, 30, 30, 32,  3,  6, 30, 33, 15, 14, 26,  8,
        7,  8,  1,  1, 23, 11, 32, 22, 16, 24,  9,  4, 25, 10, 25,  9, 23,
       32, 23, 26, 23, 26

In [105]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[

Create Neural Network Model

In [106]:
# Create model and add layers
model = Sequential()
model.add(Dense(100, input_dim=17, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(45, activation='sigmoid'))
model.add(Dense(36, activation='softmax'))

In [107]:
#View model summary

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               1800      
_________________________________________________________________
dense_9 (Dense)              (None, 36)                3636      
Total params: 5,436
Trainable params: 5,436
Non-trainable params: 0
_________________________________________________________________


Train Model

In [108]:
#Compile model
from tensorflow.keras.optimizers import SGD

opt = SGD(lr=0.01)

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [109]:
print(outcome.shape)

(1229,)


In [110]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
29/29 - 0s - loss: 0.1285 - accuracy: 0.0271
Epoch 2/100
29/29 - 0s - loss: 0.1226 - accuracy: 0.0630
Epoch 3/100
29/29 - 0s - loss: 0.1186 - accuracy: 0.1021
Epoch 4/100
29/29 - 0s - loss: 0.1154 - accuracy: 0.1401
Epoch 5/100
29/29 - 0s - loss: 0.1127 - accuracy: 0.1531
Epoch 6/100
29/29 - 0s - loss: 0.1102 - accuracy: 0.1716
Epoch 7/100
29/29 - 0s - loss: 0.1080 - accuracy: 0.1835
Epoch 8/100
29/29 - 0s - loss: 0.1061 - accuracy: 0.2041
Epoch 9/100
29/29 - 0s - loss: 0.1043 - accuracy: 0.2085
Epoch 10/100
29/29 - 0s - loss: 0.1026 - accuracy: 0.2280
Epoch 11/100
29/29 - 0s - loss: 0.1011 - accuracy: 0.2552
Epoch 12/100
29/29 - 0s - loss: 0.0997 - accuracy: 0.2562
Epoch 13/100
29/29 - 0s - loss: 0.0984 - accuracy: 0.2649
Epoch 14/100
29/29 - 0s - loss: 0.0972 - accuracy: 0.2747
Epoch 15/100
29/29 - 0s - loss: 0.0961 - accuracy: 0.2758
Epoch 16/100
29/29 - 0s - loss: 0.0950 - accuracy: 0.2780
Epoch 17/100
29/29 - 0s - loss: 0.0940 - accuracy: 0.3018
Epoch 18/100
29/29 - 0s

Evaluate Model

In [111]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.1193 - accuracy: 0.1818
Deep Neural Network - Loss: 0.11929730325937271, Accuracy: 0.1818181872367859


In [151]:
#Read in CSV and create dataframe
mlb_teams2015= pd.read_csv("2015.csv")

mlb2015_df = mlb_teams2015[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2015_LG = mlb2015_df.copy()
data2015_bin = pd.get_dummies(data2015_LG, columns=["WSWin"])
data2015_bin = data2015_bin.drop(columns=['WSWin_N'])

feature2015 = data2015_bin.drop("name", axis=1)
team2015 = data2015_bin["name"]

X_feature2015_scaler = StandardScaler().fit(feature2015)
X_feature2015_scaled = X_feature2015_scaler.transform(feature2015)

label2015_encoder = LabelEncoder()
label2015_encoder.fit(team2015)
encoded_y_2015 = label2015_encoder.transform(team2015)

y_2015_categorical = to_categorical(encoded_y_2015)

encoded_predictions2015 = model.predict_classes(X_feature2015_scaled[:10])
prediction2015_labels = label2015_encoder.transform(encoded_predictions2015)

print(f"Predicted classes: {prediction2015_labels}")
print(f"Predicted class: {model.predict_classes(test)}")

C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\numpy\lib\arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: [2, 4, 9, 11, 19, 21, 23]

In [147]:
#Read in CSV and create dataframe
mlb_teams2016= pd.read_csv("2016.csv")

mlb2016_df = mlb_teams2016[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2016_LG = mlb2016_df.copy()
data2016_bin = pd.get_dummies(data2016_LG, columns=["WSWin"])
data2016_bin = data2016_bin.drop(columns=['WSWin_N'])

feature2016 = data2016_bin.drop("name", axis=1)
team2016 = data2016_bin["name"]

X_feature2016_scaler = StandardScaler().fit(feature2016)
X_feature2016_scaled = X_feature2016_scaler.transform(feature2016)

label2016_encoder = LabelEncoder()
label2016_encoder.fit(team2016)
encoded_y_2016 = label2015_encoder.transform(team2016)

y_2016_categorical = to_categorical(encoded_y_2016)


encoded_predictions2016 = model.predict_classes(X_feature2016_scaled[:10])
#prediction2016_labels = label2016_encoder.inverse_transform(encoded_predictions2016)

print(f"Predicted classes: {encoded_predictions2016}")

Predicted classes: [11 19  3  4 21 18 14 18 32 11]


In [148]:
#Read in CSV and create dataframe
mlb_teams2017= pd.read_csv("2017.csv")

mlb2017_df = mlb_teams2017[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2017_LG = mlb2017_df.copy()
data2017_bin = pd.get_dummies(data2017_LG, columns=["WSWin"])
data2017_bin = data2017_bin.drop(columns=['WSWin_N',])

feature2017 = data2017_bin.drop("name", axis=1)
team2017 = data2017_bin["name"]

X_feature2017_scaler = StandardScaler().fit(feature2017)
X_feature2017_scaled = X_feature2017_scaler.transform(feature2017)

label2017_encoder = LabelEncoder()
label2017_encoder.fit(team2017)
encoded_y_2017 = label2017_encoder.transform(team2017)

y_2017_categorical = to_categorical(encoded_y_2017)




encoded_predictions2017 = model.predict_classes(X_feature2017_scaled[:10])
#prediction2017_labels = label2017_encoder.inverse_transform(encoded_predictions2017)

print(f"Predicted classes: {encoded_predictions2017}")

Predicted classes: [24 19  3 32 27 27 11  4 19 19]


In [149]:
#Read in CSV and create dataframe
mlb_teams2018= pd.read_csv("2018.csv")

mlb2018_df = mlb_teams2018[['yearID', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2018_LG = mlb2018_df.copy()
data2018_bin = pd.get_dummies(data2018_LG, columns=["WSWin"])
data2018_bin = data2018_bin.drop(columns=['WSWin_N'])

feature2018 = data2018_bin.drop("name", axis=1)
team2018 = data2018_bin["name"]

X_feature2018_scaler = StandardScaler().fit(feature2018)
X_feature2018_scaled = X_feature2018_scaler.transform(feature2018)

label2018_encoder = LabelEncoder()
label2018_encoder.fit(team2018)
encoded_y_2018 = label2018_encoder.transform(team2018)

y_2018_categorical = to_categorical(encoded_y_2018)


encoded_predictions2018 = model.predict_classes(X_feature2018_scaled[:10])
#prediction2018_labels = label2018_encoder.inverse_transform(encoded_predictions2018)

print(f"Predicted classes: {encoded_predictions2018}")

Predicted classes: [24 19 11 33 21 25  2 22 16  6]


In [150]:
#Read in CSV and create dataframe
mlb2019_teams= pd.read_csv("MLB 2019 season.csv")



mlb2019_df = mlb2019_teams[['yearID','W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]


data2019_LG = mlb2019_df.copy()
data2019_bin = pd.get_dummies(data2019_LG, columns=["WSWin"])
data2019_bin = data2019_bin.drop(columns=['WSWin_N'])



feature2019 = data2019_bin.drop("name", axis=1)
team2019 = data2019_bin["name"]


X_feature2019_scaler = StandardScaler().fit(feature2019)
X_feature2019_scaled = X_feature2019_scaler.transform(feature2019)


label_team_encoder2019 = LabelEncoder()
label_team_encoder2019.fit(team)
encoded_y_team201 = label_team_encoder2019.transform(team2019)


y_team2019_categorical = to_categorical(encoded_y_team)

encoded_predictions2019 = model.predict_classes(X_feature2019_scaled[:10])
#prediction2019_labels = label_team_encoder2019.inverse_transform(encoded_predictions2019)

print(f"Predicted classes: {encoded_predictions2019}")


Predicted classes: [21 27 26  4 19 27 16  3 11 25]


In [91]:
encoded_predictions = model.predict_classes(X_feature2019_scaled[:30])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Predict World Series Champs

In [92]:
print(f"Predicted classes: {prediction_labels}")


Predicted classes: ['San Diego Padres' 'Philadelphia Phillies' 'Detroit Tigers'
 'Boston Red Sox' 'Minnesota Twins' 'Chicago Cubs' 'Chicago White Sox'
 'Baltimore Orioles' 'Minnesota Twins' 'Arizona Diamondbacks'
 'New York Yankees' 'Florida Marlins' 'Atlanta Braves' 'Oakland Athletics'
 'Philadelphia Phillies' 'Cincinnati Reds' 'Atlanta Braves'
 'Baltimore Orioles' 'Baltimore Orioles' 'New York Yankees'
 'Boston Red Sox' 'Kansas City Royals' 'San Diego Padres'
 'Oakland Athletics' 'Pittsburgh Pirates' 'Oakland Athletics'
 'San Diego Padres' 'Philadelphia Phillies' 'Milwaukee Brewers'
 'St. Louis Cardinals']


In [28]:
import joblib
filename = 'SWilliams_SVM.sav'
joblib.dump(model, filename)

TypeError: can't pickle _thread.RLock objects